In [2]:
import numpy as n
import pygame

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [13]:
class Particle():
    def __init__(self, x, y, width, height, vx=0, vy=0):
        self.pos = pygame.Vector2(x, y)
        self.width = width
        self.height = height
        self.vel = pygame.Vector2(vx, vy)
        self.rect = pygame.Rect(self.pos.x, self.pos.y, self.width, self.height)
        
        self.updated = False
        
    def update(self):
        pass
    
    def collide(self, particle):
        pass

In [4]:
p = Particle(100, 100, 1, 1)
p.rect

<rect(100, 100, 1, 1)>

In [5]:
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 800
NBOX_X = 10
NBOX_Y = 10
BOX_WIDTH = SCREEN_WIDTH // NBOX_X
BOX_HEIGHT = SCREEN_HEIGHT // NBOX_Y

class Box():
    def __init__(self, x, y, w, h, index):
        self.rect = pygame.Rect(x, y, w, h)
        self.index = index
        self.particles = []
        
    def updateBox(self):
        # start in first box
        # step 1: get all particles in the box
        # step 2: check for collisions between particles in this box and particles in neighboring boxes
        # step 3: call collision handler for each collision (method of particle class)
        # step 4: update particles in this box
        # step 5: go to next box
        
        neighbors = []
        if 
        
    def checkParticles(self):
        for particle in self.particles:
            box_x, box_y, box_w, box_h = self.rect
            x, y, w, h = particle.rect
            
            if x-w > box_x+box_w or x+w < box_x or y+h < box_y or y-h > box_y+box_h:
                self.removeParticle(particle)
            
            if x+w > box_x+box_w:
                boxes[self.index+1].addParticle(particle)
            if x-w < box_x:
                boxes[self.index-1].addParticle(particle)
            if y+h > box_y+box_h:
                boxes[self.index+NBOX_X].addParticle(particle)
            if y-h < box_y:
                boxes[self.index-NBOX_X].addParticle(particle)
                
            
            self.updateParticles()
        
    def addParticle(self, particle):
        if not particle.updated and particle not in self.particles:
            self.particles.append(particle)
            particle.updated = True
        
    def removeParticle(self, particle):
        self.particles.remove(particle)
        
    def updateParticles(self):
        for particle in self.particles:
            particle.update()
            
    def contains_particles(self):
        return len(self.particles) > 0
        
    def draw(self, screen, color=(255, 255, 255), signal_content=False):
        if signal_content and self.contains_particles():
                color = (200, 120, 120)
        pygame.draw.rect(screen, color, self.rect)
        

boxes = []
for j in range(NBOX_Y):
    for i in range(NBOX_X):
        boxes.append(Box(i*BOX_WIDTH, j*BOX_HEIGHT, BOX_WIDTH, BOX_HEIGHT))

        
        
class Boxes():
    def __init__(self, screen_width, screen_height, nbox_x, nbox_y, particles=None):
        self.nbox_x = nbox_x
        self.nbox_y = nbox_y
        self.box_w = screen_width // nbox_x
        self.box_h = screen_height // nbox_y
        self.particles = particles
        
        if self.box_w * nbox_x != screen_width:
            raise ValueError("Screen width is not a multiple of nbox_x")
        if self.box_h * nbox_y != screen_height:
            raise ValueError("Screen height is not a multiple of nbox_y")
        
        self.boxes = []
        for j in range(nbox_y):
            for i in range(nbox_x):
                self.boxes.append(Box(i*self.box_w, j*self.box_h, self.box_w, self.box_h))
        
    def updateBoxes(self):
        for box in self.boxes:
            box.removeParticles()
            particle_indices = box.rect.collidelistall(self.particles)
            if particle_indices != -1:
                box.addParticles(self.particles, particle_indices)
                
            box.updateParticles()
        
    def draw(self, screen, signal_content=False):
        for box in self.boxes:
            box.draw(screen, (200, 200, 200), signal_content)
            
    def addParticle(self, particle):
        for box in self.boxes:
            if box.rect.colliderect(particle.rect):
                box.addParticle(particle)
                
    def removeParticle(self, particle):
        for box in self.boxes:
            if not box.rect.colliderect(particle.rect):
                box.removeParticle(particle)

In [11]:
# starting pygame
pygame.init()

# setting up the Screen
width = 800
height = 600

# making the boxes
boxes = Boxes(width, height, 8, 6)

# making particle
p = Particle(95, 100, 20, 20)
p1 = Particle(399, 199, 60, 60)
boxes.addParticle(p)
boxes.addParticle(p1)

# setting up the screen
screen = pygame.display.set_mode((width, height))

# setting up the clock
clock = pygame.time.Clock()

# setting up the game loop
running = True
while running:
    screen.fill((0, 0, 0))
    
    boxes.draw(screen, signal_content=True)
    pygame.draw.rect(screen, (250,250,250), p.rect)
    pygame.draw.rect(screen, (250,250,250), p1.rect)
        
    
    
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    
    pygame.display.flip()
    clock.tick(60)

KeyboardInterrupt: 